# MVP: Modelagem de Ameaças com IA (STRIDE)

Este notebook cria um MVP para detectar componentes em diagramas de arquitetura e gerar ameaças e contramedidas baseadas em STRIDE.

## 1) Setup

In [ ]:
!apt-get -qq update && apt-get -qq install -y tesseract-ocr
!pip -q install ultralytics opencv-python pytesseract

In [ ]:
from pathlib import Path
import zipfile
from google.colab import files
import cv2
import pytesseract
from ultralytics import YOLO

In [ ]:
# Clona/atualiza o repositório e garante download via Git LFS
import os
import glob
import subprocess

DIR_REPO = '/content/fiap-fase-5-hackaton'
if not os.path.isdir(DIR_REPO):
    !git clone https://github.com/patrickpacifico/fiap-fase-5-hackaton.git
else:
    !git -C /content/fiap-fase-5-hackaton pull

# Garante git-lfs instalado
try:
    subprocess.run(['git', 'lfs', 'install'], check=True)
except Exception:
    !apt-get -qq update && apt-get -qq install -y git-lfs
    !git lfs install

# Baixa arquivos LFS
!git -C /content/fiap-fase-5-hackaton lfs pull

print('Arquivos no repo:')
!ls /content/fiap-fase-5-hackaton

# Procura o zip no repo
encontrados = glob.glob('/content/fiap-fase-5-hackaton/**/arqsof_dataset.zip', recursive=True)
print('Zips encontrados:', encontrados)

CAMINHO_ZIP = encontrados[0] if encontrados else ''
print('CAMINHO_ZIP definido como:', CAMINHO_ZIP)


## 2) Escolha do modo
Use `treino` quando tiver dataset anotado (YOLO/COCO).
Use `inferencia` quando tiver apenas uma imagem do diagrama.

In [ ]:
MODO = 'inferencia'  # 'treino' ou 'inferencia'
print('Modo selecionado:', MODO)

## 3A) Upload do dataset (somente treino)
Envie um .zip contendo o dataset. Estruturas aceitas:
- YOLO padrão: images/train, images/val, labels/train, labels/val
- YOLO sem splits: images/ e labels/ (o notebook cria train/val)
- VOC: imagens + arquivos .xml (o notebook converte para YOLO)
- Pasta única com imagens + .txt (o notebook organiza em YOLO)

Exemplo comum: `src/dataset/dataset_augmented` com arquivos `.png/.jpg` e `.xml`.

In [ ]:
# Upload do dataset ou uso do zip localizado no repo
if MODO != 'treino':
    print('Pulando upload de dataset (modo não é treino).')
else:
    if CAMINHO_ZIP:
        NOME_ZIP = CAMINHO_ZIP
        print('Usando zip local:', NOME_ZIP)
    else:
        ARQUIVOS = files.upload()
        NOME_ZIP = next(iter(ARQUIVOS))

    DIR_EXTRACAO = Path('/content/dataset_zip')
    DIR_EXTRACAO.mkdir(parents=True, exist_ok=True)

    with zipfile.ZipFile(NOME_ZIP, 'r') as zf:
        zf.extractall(DIR_EXTRACAO)

    print('Dataset extraído em:', DIR_EXTRACAO)

In [ ]:
# Preparação automática do dataset para YOLO (suporta VOC .xml)
import random
import shutil
import xml.etree.ElementTree as ET
from pathlib import Path

EXTENSOES_IMAGEM = {'.jpg', '.jpeg', '.png', '.bmp'}

def possui_estrutura_yolo(dir_base):
    return all((dir_base / p).exists() for p in [
        'images/train', 'images/val', 'labels/train', 'labels/val'
    ])

def possui_images_labels_sem_split(dir_base):
    return (dir_base / 'images').exists() and (dir_base / 'labels').exists()

def listar_imagens(pasta):
    return [p for p in pasta.rglob('*') if p.suffix.lower() in EXTENSOES_IMAGEM]

def encontrar_pasta_com_pares(dir_base):
    melhores = (0, None)
    for pasta in [dir_base] + [p for p in dir_base.rglob('*') if p.is_dir()]:
        imgs = [p for p in pasta.iterdir() if p.is_file() and p.suffix.lower() in EXTENSOES_IMAGEM]
        if not imgs:
            continue
        pares = [img for img in imgs if (pasta / f'{img.stem}.txt').exists()]
        if len(pares) > melhores[0]:
            melhores = (len(pares), pasta)
    return melhores[1]

def coletar_xmls_recursivo(dir_base):
    return [p for p in Path(dir_base).rglob('*') if p.is_file() and p.suffix.lower() == '.xml']

def mapear_imagens_recursivo(dir_base):
    imagens = listar_imagens(dir_base)
    por_stem = {}
    por_nome = {}
    for img in imagens:
        por_stem.setdefault(img.stem, img)
        por_nome.setdefault(img.name, img)
    return imagens, por_stem, por_nome

def criar_estrutura_yolo(pares, dir_destino, proporcao_treino=0.8, seed=42):
    random.seed(seed)
    random.shuffle(pares)
    corte = int(len(pares) * proporcao_treino)
    treino, validacao = pares[:corte], pares[corte:]

    (dir_destino / 'images/train').mkdir(parents=True, exist_ok=True)
    (dir_destino / 'images/val').mkdir(parents=True, exist_ok=True)
    (dir_destino / 'labels/train').mkdir(parents=True, exist_ok=True)
    (dir_destino / 'labels/val').mkdir(parents=True, exist_ok=True)

    def copiar(pares, split):
        for img, label in pares:
            shutil.copy2(img, dir_destino / f'images/{split}/{img.name}')
            shutil.copy2(label, dir_destino / f'labels/{split}/{label.name}')

    copiar(treino, 'train')
    copiar(validacao, 'val')

def ler_voc(xml_path):
    root = ET.parse(xml_path).getroot()
    size = root.find('size')
    w = int(size.find('width').text)
    h = int(size.find('height').text)
    objetos = []
    for obj in root.findall('object'):
        nome = obj.find('name').text.strip()
        bbox = obj.find('bndbox')
        xmin = float(bbox.find('xmin').text)
        ymin = float(bbox.find('ymin').text)
        xmax = float(bbox.find('xmax').text)
        ymax = float(bbox.find('ymax').text)
        x = ((xmin + xmax) / 2) / w
        y = ((ymin + ymax) / 2) / h
        bw = (xmax - xmin) / w
        bh = (ymax - ymin) / h
        objetos.append((nome, x, y, bw, bh))
    return objetos

def obter_nome_arquivo_voc(xml_path):
    try:
        root = ET.parse(xml_path).getroot()
        filename = root.find('filename')
        if filename is not None and filename.text:
            return Path(filename.text.strip()).name
    except Exception:
        pass
    return None

def coletar_classes_voc(xmls):
    classes = []
    vistos = set()
    for xml in xmls:
        for nome, *_ in ler_voc(xml):
            if nome not in vistos:
                vistos.add(nome)
                classes.append(nome)
    return classes

def encontrar_imagem_para_xml(xml_path, por_stem, por_nome):
    stem = xml_path.stem
    if stem in por_stem:
        return por_stem[stem]
    nome_arquivo = obter_nome_arquivo_voc(xml_path)
    if nome_arquivo and nome_arquivo in por_nome:
        return por_nome[nome_arquivo]
    return None

def converter_voc_para_yolo(xmls, dir_destino, por_stem, por_nome, proporcao_treino=0.8, seed=42, nomes_classes=None):
    pares = []
    for xml in xmls:
        img = encontrar_imagem_para_xml(xml, por_stem, por_nome)
        if img is not None:
            pares.append((img, xml))

    if not pares:
        raise FileNotFoundError('Nenhuma imagem correspondente aos XMLs foi encontrada.')

    if not nomes_classes:
        nomes_classes = coletar_classes_voc([xml for _, xml in pares])

    mapa = {nome: i for i, nome in enumerate(nomes_classes)}

    random.seed(seed)
    random.shuffle(pares)
    corte = int(len(pares) * proporcao_treino)
    treino, validacao = pares[:corte], pares[corte:]

    (dir_destino / 'images/train').mkdir(parents=True, exist_ok=True)
    (dir_destino / 'images/val').mkdir(parents=True, exist_ok=True)
    (dir_destino / 'labels/train').mkdir(parents=True, exist_ok=True)
    (dir_destino / 'labels/val').mkdir(parents=True, exist_ok=True)

    def processar(pares, split):
        for img, xml in pares:
            shutil.copy2(img, dir_destino / f'images/{split}/{img.name}')
            objetos = ler_voc(xml)
            linhas = []
            for nome, x, y, bw, bh in objetos:
                if nome not in mapa:
                    continue
                linhas.append(f'{mapa[nome]} {x:.6f} {y:.6f} {bw:.6f} {bh:.6f}')
            label_path = dir_destino / f'labels/{split}/{img.stem}.txt'
            label_path.write_text('\n'.join(linhas), encoding='utf-8')

    processar(treino, 'train')
    processar(validacao, 'val')

    return dir_destino, nomes_classes

def preparar_dataset_yolo(dir_base):
    if possui_estrutura_yolo(dir_base):
        return dir_base, None

    if possui_images_labels_sem_split(dir_base):
        imgs = listar_imagens(dir_base / 'images')
        pares = []
        for img in imgs:
            label = (dir_base / 'labels' / f'{img.stem}.txt')
            if label.exists():
                pares.append((img, label))
        if not pares:
            raise FileNotFoundError('Nenhum par imagem/label encontrado em images/ e labels/.')

        dir_yolo = Path('/content/data_yolo')
        criar_estrutura_yolo(pares, dir_yolo)
        return dir_yolo, None

    xmls = coletar_xmls_recursivo(dir_base)
    if xmls:
        imagens, por_stem, por_nome = mapear_imagens_recursivo(dir_base)
        print(f'Total de XMLs encontrados: {len(xmls)}')
        print(f'Total de imagens encontradas: {len(imagens)}')
        dir_yolo = Path('/content/data_yolo')
        nomes_classes = None
        if 'NOMES_CLASSES' in globals() and NOMES_CLASSES:
            nomes_classes = NOMES_CLASSES
        dir_yolo, classes_detectadas = converter_voc_para_yolo(
            xmls, dir_yolo, por_stem, por_nome, nomes_classes=nomes_classes
        )
        print('Classes detectadas no VOC:', classes_detectadas)
        return dir_yolo, classes_detectadas

    pasta_pares = encontrar_pasta_com_pares(dir_base)
    if pasta_pares is None:
        raise FileNotFoundError('Nenhuma pasta com imagens e labels (.txt/.xml) encontrada.')

    imgs = [p for p in pasta_pares.iterdir() if p.is_file() and p.suffix.lower() in EXTENSOES_IMAGEM]
    pares = []
    for img in imgs:
        label = (pasta_pares / f'{img.stem}.txt')
        if label.exists():
            pares.append((img, label))

    if not pares:
        raise FileNotFoundError('Nenhum par imagem/label encontrado na pasta detectada.')

    dir_yolo = Path('/content/data_yolo')
    criar_estrutura_yolo(pares, dir_yolo)
    return dir_yolo, None

def gerar_data_yaml(dir_base, nomes_classes):
    caminho = dir_base / 'data.yaml'
    linhas = [
        f'path: {dir_base}',
        'train: images/train',
        'val: images/val',
        '',
        'names:'
    ]
    for i, nome in enumerate(nomes_classes):
        linhas.append(f'  {i}: {nome}')
    caminho.write_text('\n'.join(linhas), encoding='utf-8')
    return caminho

if MODO == 'treino':
    DIR_DATASET_TREINO, CLASSES_DETECTADAS = preparar_dataset_yolo(DIR_EXTRACAO)
    print('Dataset preparado em:', DIR_DATASET_TREINO)

    NOMES_CLASSES = []
    if 'CLASSES_DETECTADAS' in globals() and CLASSES_DETECTADAS:
        NOMES_CLASSES = CLASSES_DETECTADAS
    else:
        NOMES_CLASSES = [
            'user', 'server', 'database', 'api', 'gateway', 'queue',
            'storage', 'third_party', 'auth', 'frontend', 'backend', 'network'
        ]

    CAMINHO_DATA_YAML = gerar_data_yaml(DIR_DATASET_TREINO, NOMES_CLASSES)
    print('data.yaml gerado em:', CAMINHO_DATA_YAML)

## 3B) Upload de imagem única (somente inferência)
Envie uma imagem (.png ou .jpg) do diagrama de arquitetura.

In [ ]:
if MODO != 'inferencia':
    print('Pulando upload de imagem (modo não é inferência).')
else:
    ARQUIVOS_IMG = files.upload()
    NOME_IMG = next(iter(ARQUIVOS_IMG))
    CAMINHO_IMAGEM = str(Path(NOME_IMG).resolve())
    print('Imagem carregada:', CAMINHO_IMAGEM)

## 4) Treinamento do detector (YOLOv8) — opcional

In [ ]:
PESOS_BASE = 'yolov8n.pt'
modelo = YOLO(PESOS_BASE)

if MODO == 'treino':
    if 'CAMINHO_DATA_YAML' not in globals():
        raise ValueError('CAMINHO_DATA_YAML não definido. Verifique a preparação do dataset.')
    # modelo.train(data=CAMINHO_DATA_YAML, epochs=50, imgsz=640)


## 5) Inferência

In [ ]:
if MODO == 'inferencia':
    PESOS_INFERENCIA = 'yolov8n.pt'  # ou '/content/runs/detect/train/weights/best.pt'
    modelo = YOLO(PESOS_INFERENCIA)

    resultados = modelo.predict(source=CAMINHO_IMAGEM, conf=0.25)
    deteccoes = []
    for r in resultados:
        for c in r.boxes.cls.tolist():
            deteccoes.append(modelo.names[int(c)])

    MAPA_COMPONENTES = {
        'user': 'usuário',
        'server': 'servidor',
        'database': 'base de dados',
        'api': 'api',
        'gateway': 'gateway',
        'queue': 'fila',
        'storage': 'armazenamento',
        'third_party': 'terceiros',
        'auth': 'autenticação',
        'frontend': 'frontend',
        'backend': 'backend',
        'network': 'rede'
    }

    deteccoes_pt = [MAPA_COMPONENTES.get(d, d) for d in deteccoes]
    print('Detecções (bruto):', deteccoes)
    print('Detecções (PT-BR):', deteccoes_pt)


In [ ]:
# Visualização das detecções
import matplotlib.pyplot as plt

if 'resultados' in globals() and resultados:
    img_plot = resultados[0].plot()
    img_plot = cv2.cvtColor(img_plot, cv2.COLOR_BGR2RGB)

    plt.figure(figsize=(10, 10))
    plt.imshow(img_plot)
    plt.axis('off')
else:
    print('Sem resultados para visualizar.')


## 6) Mapeamento STRIDE

In [ ]:
MAPA_COMPONENTES = {
    'user': 'usuário',
    'server': 'servidor',
    'database': 'base de dados',
    'api': 'api',
    'gateway': 'gateway',
    'queue': 'fila',
    'storage': 'armazenamento',
    'third_party': 'terceiros',
    'auth': 'autenticação',
    'frontend': 'frontend',
    'backend': 'backend',
    'network': 'rede'
}

MAPA_STRIDE = {
    'user': ['Falsificação de identidade', 'Elevação de privilégio'],
    'auth': ['Falsificação de identidade', 'Elevação de privilégio', 'Repúdio'],
    'api': ['Violação (manipulação)', 'Repúdio', 'Negação de serviço'],
    'gateway': ['Violação (manipulação)', 'Repúdio', 'Negação de serviço'],
    'database': ['Divulgação de informação', 'Violação (manipulação)'],
    'storage': ['Divulgação de informação', 'Violação (manipulação)'],
    'queue': ['Violação (manipulação)', 'Negação de serviço'],
    'frontend': ['Falsificação de identidade', 'Violação (manipulação)', 'Divulgação de informação'],
    'backend': ['Violação (manipulação)', 'Repúdio', 'Negação de serviço'],
    'network': ['Falsificação de identidade', 'Divulgação de informação', 'Negação de serviço'],
    'server': ['Violação (manipulação)', 'Negação de serviço', 'Elevação de privilégio'],
    'third_party': ['Falsificação de identidade', 'Divulgação de informação', 'Repúdio']
}

def mapear_stride(componentes):
    saida = {}
    for c in componentes:
        saida[c] = MAPA_STRIDE.get(c, [])
    return saida

## 7) Geração de relatório

In [ ]:
componentes = []
if 'deteccoes' in globals() and deteccoes:
    componentes = deteccoes
else:
    componentes = ['api', 'database', 'user']

mapeamento = mapear_stride(componentes)

linhas = ['# Relatório de Modelagem de Ameaças', '']
for comp in componentes:
    comp_label = MAPA_COMPONENTES.get(comp, comp)
    ameacas = mapeamento.get(comp, [])
    linhas.append(f'## {comp_label}')
    if ameacas:
        for a in ameacas:
            linhas.append(f'- {a}')
    else:
        linhas.append('- Nenhuma ameaça mapeada')
    linhas.append('')

relatorio = '\n'.join(linhas)
print(relatorio)

with open('relatorio.md', 'w', encoding='utf-8') as f:
    f.write(relatorio)